In [14]:
from pyspark.sql import SparkSession
import timeit 

experiment_1 = {'cores': 2, 'dataset': "RC_2005-12.json"}
experiment_2 = {'cores': 2, 'dataset': "RC_2009-05"}
experiment_3 = {'cores': 5, 'dataset': "RC_2011-08.json"}

config = experiment_3
# New API
spark_session = SparkSession\
        .builder\
        .master("spark://192.168.2.131:7077") \
        .appName("Team7_code.ipynb")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.shuffle.service.enabled", True)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.cores.max",config['cores'])\
        .config("spark.worker.instances",5)\
        .config("spark.executor.cores",1)\
        .config("spark.driver.port",9998)\
        .config("spark.blockManager.port",10005)\
        .getOrCreate()
#\#)\
# Old API (RDD)
spark_context = spark_session.sparkContext

spark_context.setLogLevel("INFO")
print("Started Session")

Started Session


Data loading

In [16]:
def create_spark_dataframe(filename):
    df = spark_session.read.json('hdfs://192.168.2.131:9000/user/ubuntu/{}'.format(filename))
    return df

start_time_0 = timeit.default_timer() 

#df = create_spark_dataframe('RC_2005-12.json')
#df = create_spark_dataframe('RC_2011-08.json')
#df = create_spark_dataframe('RC_2009-05.json')
df = create_spark_dataframe(config['dataset'])
                            
print("execution time {}s".format(timeit.default_timer()-start_time_0))

execution time 91.78741989197442s


Author comments count

In [11]:
from pyspark.sql.functions import desc, col
start_time_0 = timeit.default_timer() 

author_comment_counts = df.groupBy('author').count()
author_comment_counts.cache()
author_comment_counts.sort(desc("count")).show()
print("execution time {}s".format(timeit.default_timer()-start_time_0))

+---------------+------+
|         author| count|
+---------------+------+
|      [deleted]|359647|
|      karmanaut|  1594|
|            db2|  1394|
|ReligionOfPeace|  1087|
|      randomb0y|  1087|
|     malcontent|  1074|
|  CliffDropOver|  1043|
|TruthinessHurts|  1042|
|        defrost|   992|
|    rogerssucks|   983|
|     AnnArchist|   945|
|      crackduck|   883|
|        IConrad|   877|
|     linkedlist|   846|
|       masklinn|   839|
|    eaturbrainz|   824|
|     liquidpele|   824|
|      glengyron|   804|
|     itstallion|   797|
|         ropers|   790|
+---------------+------+
only showing top 20 rows

execution time 15.21671885298565s


Word count and score

In [17]:
import re
def pre_process_text(text):
    text = text.lower()
    text = text.replace('\r', '')
    text = text.replace('\n', ' ')
    text = text.strip()
    text=text.split(' ')
    text = map(lambda x: re.sub(r'[^a-zA-Z0-9]+', '', x), text)
    text = filter(lambda x: len(x) > 0, text)
    return text

total_time = timeit.default_timer() 

start_time_0 = timeit.default_timer() 
#author_comment_counts.select("count").rdd.map(tuple).map(lambda x: x[0]).map(lambda x: x**2).collect()
df_tmp = df.select(['body','score']).rdd.map(tuple).map(lambda x: (pre_process_text(x[0]), x[1])).flatMap(lambda x: [(y, x[1]) for y in x[0]]).cache()
word_counts =df_tmp.map(lambda x: (x[0], 1)).reduceByKey(lambda x,y: x+y)
print(word_counts.sortBy(lambda x: x[1], ascending=False).take(10))
print("execution time {}s".format(timeit.default_timer()-start_time_0))

start_time_0 = timeit.default_timer() 
word_scores = df_tmp.reduceByKey(lambda x,y: x+y)
print(word_scores.sortBy(lambda x: x[1], ascending=False).take(10))
print("execution time {}s".format(timeit.default_timer()-start_time_0))

start_time_0 = timeit.default_timer() 
word_avg_score = word_counts.filter(lambda x: x[1]>5).join(word_scores).map(lambda x: (x[0], x[1][1]/x[1][0]))
print(word_avg_score.sortBy(lambda x: x[1], ascending=False).take(100))
print("execution time {}s".format(timeit.default_timer()-start_time_0))

print("Total execution time {}s".format(timeit.default_timer()-total_time))

----------------------------------------ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/home/ubuntu/.local/lib/python3.8/site-packages/py4j/java_gateway.py", line 1207, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ubuntu/.local/lib/python3.8/site-packages/py4j/java_gateway.py", line 1033, in send_command
    response = connection.send_command(command)
  File "/home/ubuntu/.local/lib/python3.8/site-packages/py4j/java_gateway.py", line 1211, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while receiving

Exception happened during processing of request from ('127.0.0.1', 44538)ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:39617)
Traceback (most recent call

Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:39617)

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:39617)
Traceback (most recent call last):
  File "/usr/lib/python3.8/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.8/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.8/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.8/socketserver.py", line 720, in __init__
    self.handle()
  File "/home/ubuntu/.local/lib/python3.8/site-packages/pyspark/accumulators.py", line 262, in handle
    poll(accum_updates)
  File "/home/ubuntu/.local/lib/python3.8/site-packages/pyspark/accumulators.py", line 235, in poll
    if func():
  File "/home/ubuntu/.local/lib/python3.8/site-packages/pyspark/accumulators.py", line 239, in accum_updates
    num_updates = read_int(self.rfile)


In [12]:
# release the cores for another application!
spark_context.stop()
print("Ended Session")

Ended Session
